# Create boundry conditions from the Jena Carboscope fluxes

We model the contribution from boundary conditions at the edge of our domain using fluxes from the Jena Carboscope inversion

We calculate the boundary conditions for APO, CO2, and O2

### Import modules

In [19]:
import os, glob
from datetime import date

from acrg.name import name
from acrg.BC import timevarying_BC
from acrg.config.paths import Paths

### Define inputs

In [37]:
years = [2010, 2011]
species = ['apo', 'co2']

adjust = {'apo': -18.0, 'co2': 310.052}

## Create boundary conditions

### Find data file

In [38]:
jena_file_names = {spec: {year: f'apo99X_WAO_v2021_mix_{year}.nc' if spec=='apo' else \
                                f's10oc_v2021_mix_{year}.nc'
                          for year in years}
                   for spec in species}
jena_files = {spec: {year: os.path.join(Paths.data, 'Gridded_fluxes', spec.upper(), 'Jena_Carboscope', jena_file_name)
                     for year, jena_file_name in jena_spec.items()}
              for spec, jena_spec in jena_file_names.items()}

for spec, jena_spec in jena_files.items():
    for year, jena_file in jena_spec.items():
        if not os.path.isfile(jena_file):
            print(f'File not found: {jena_file}')
        else:
            print(f'File found: {jena_file}')

File found: /group/chemistry/acrg/Gridded_fluxes/APO/Jena_Carboscope/apo99X_WAO_v2021_mix_2010.nc
File found: /group/chemistry/acrg/Gridded_fluxes/APO/Jena_Carboscope/apo99X_WAO_v2021_mix_2011.nc
File found: /group/chemistry/acrg/Gridded_fluxes/CO2/Jena_Carboscope/s10oc_v2021_mix_2010.nc
File found: /group/chemistry/acrg/Gridded_fluxes/CO2/Jena_Carboscope/s10oc_v2021_mix_2011.nc


### Create boundary conditions file

In [39]:
out_path = {spec: os.path.join(Paths.lpdm, 'bc', 'EUROPE', f'{spec}_JenaCarboscope') \
                  if spec in ['co2'] else \
                  os.path.join(Paths.lpdm, 'bc', 'EUROPE')
            for spec in species}

for spec, jena_spec in jena_files.items():
    print(f'Saving {spec} bcs to dir: {out_path[spec]}')

    if not os.path.isdir(out_path[spec]):
        os.mkdir(out_path[spec])

    for year, jena_file in jena_spec.items():
        print(f'{year}, raw file: {jena_file}')
        if os.path.isfile(jena_file):
            bc_obj = timevarying_BC.BoundaryConditions(vmr_var        = f'{spec}mix',
                                                       gph_height_var = 'gph',
                                                       filename       = jena_file,
                                                       time_coord     = 'time',
                                                       species        = spec.upper(),
                                                       domain         = 'EUROPE',
                                                       start_date     = None,
                                                       adjust         = adjust[spec])

            bc_obj.make_bc_file(fp_directory    = None,
                                fp_height_coord = 'height',
                                reverse         = None,
                                convert_units   = False,
                                datasource      = 'JenaCarboscope',
                                out_path        = out_path[spec],
                                glob_attrs      = {},
                                copy_glob_attrs = ['history', 'references'],
                                verbose         = True)
        else:
            print(f'File not found:\n{jena_file}')

Saving apo bcs to dir: /group/chemistry/acrg/LPDM/bc/EUROPE
2010, Raw file: /group/chemistry/acrg/Gridded_fluxes/APO/Jena_Carboscope/apo99X_WAO_v2021_mix_2010.nc
Adjusting vmr by -18.0

Cutting vmr data to edges
-------------------------
Using footprint file: /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/MHD-10magl_UKV_hfo-1234yf_EUROPE_201404.nc to extract domain

Interpolating
-------------
-- Interpolating north boundary vmr --
Interpolating height along lon
Interpolating lat/lon along lon
-- Interpolating south boundary vmr --
Interpolating height along lon
Interpolating lat/lon along lon
-- Interpolating east boundary vmr --
Interpolating height along lat
Interpolating lat/lon along lat
-- Interpolating west boundary vmr --
Interpolating height along lat
Interpolating lat/lon along lat

Saving to netcdf
----------------
Copying attribute history
Copying attribute references
Output filename : apo_EUROPE_201001.nc
Saving boundary conditions to : /group/chemistry/acrg/LPDM/bc/EUROPE/apo_

## Calcuate the O2 boundary conditions

We can calculate the O2 boundary conditions from the APO and CO2

Information to be used below:

Reference values are taken from Rodenbeck et al., 2008:

$X_0^{CO_2} = 350$ \
$X_0^{N_2} = 790190$ \
$X_0^{O_2} = 209460$

In [ ]:
ref_values = {'co2': 350, 'n2': 790190, 'o2': 209460}
ref_values['o2_n2'] = ref_values['o2'] / ref_values['n2']

## Get the data

Get the netcdf files for the apo and co2 boundary conditions and open using xarray

In [ ]:
bc_dir = {'apo': os.path.join(Paths.lpdm, 'bc', 'EUROPE')}
bc_dir['co2'] = os.path.join(bc_dir['apo'], 'co2_JenaCarboscope')

bc_files = {spec: glob.glob(os.path.join(directory, f'{spec}_EUROPE_{year}01.nc'))[0]
            for spec, directory in bc_dir.items()}

[print(f'{spec}: {bc}') for spec, bc in bc_files.items()]

bc_data = {spec: name.open_ds(bc) for spec, bc in bc_files.items()}

In [ ]:
bc_data['apo'] = bc_data['apo'] / 0.20946 if (abs(bc_data['apo'].vmr_n)<100).all() else bc_data['apo']
bc_data['apo']

## Calculate the O2 boundary conditions

Use the APO equation to calculate the $\delta (O2/N2)$ boundary conditions:

$\delta (APO) = \delta (O2/N2) + 1.1 / X_O^{O2} (CO_2 - X_0^{CO_2}) \times 10^6$

\
Then use the $\delta (O2/N2)$ equation to calculate the O2 boundary conditions:

$\delta (O2/N2) = \frac{(O_2 / N_2)_{sample} - (O_2 / N_2)_{refence}}{(O_2 / N_2)_{refence}}$

In [ ]:
per_meg_conversion = 1e6/ref_values['o2']
print(f'per meg conversion: {per_meg_conversion}')
del_o2_n2 = bc_data['apo'] * per_meg_conversion - 1.1 * per_meg_conversion * (bc_data['co2'] - ref_values['co2'])
bc_o2 = (del_o2_n2 * ref_values['o2_n2'] * 1e-6 + ref_values['o2_n2']) * ref_values['n2']

## Save to netcdf

Add attributes to the O2 dataset including the sim from which the bcs are estimated and the date created.

Create a standard filename in the same format as those for other species in /user/work/chxmr/shared/LPDM/bc/EUROPE .\
Save the dataset to a netcdf file in the shared directory.

In [ ]:
# copy the CO2 attributes
o2_attrs = bc_data['co2'].attrs
# edit the species in the attributes title
o2_attrs['title'] = 'O2'.join(bc_data['co2'].attrs['title'].split('CO2'))
# add a description to explain how the bcs were estimated
o2_attrs['description'] = 'Estimated from Jena Carboscope CO2 and APO boundary conditions using APO equation'
# edit the creation date
o2_attrs['date_created'] = str(date.today())

# add the attributes to the O2 dataset
bc_o2 = bc_o2.assign_attrs(o2_attrs)

# create a standardised filename
filename = os.path.join(bc_dir['apo'], f'o2_EUROPE_{year}01.nc')

if os.path.isfile(filename):
    print(f'Boundary condition file {filename} already exists.')
    answer = input("You are about to overwrite an existing file, do you want to continue? Y/N ")
    save = False if answer.upper() == 'N' else True
else:
    save = True

# save the file
if save:
    print(f'Saving to: {filename}')
    bc_o2.to_netcdf(filename)
else:
    print('Not overwriting file')